In [ ]:
%cd ~/Dev/video-membership

In [ ]:
from app import db
from app.playlists.models import Playlist
from app.users.models import User
from app.videos.models import Video

from cassandra.cqlengine.management import sync_table

In [ ]:
from app.indexing.client import get_index, update_index
from app.indexing.schemas import *

In [ ]:
db.get_session()
sync_table(User)
sync_table(Video)
sync_table(Playlist)

In [ ]:
import uuid
import json
from pydantic import BaseModel, Field, validator, root_validator
from typing import Optional

In [ ]:
video_obj = dict(Video.objects.first())
video_obj

In [ ]:
class VideoIndexSchema2(BaseModel):
    objectID: str = Field(alias='host_id')
    objectType: str = "Video"
    title: Optional[str]
    path: str = Field(alias='host_id')
        
    @validator("path")
    def set_path(cls, v, values, **kwargs):
        host_id = v
        return f"/videos/{host_id}"

In [ ]:
VideoIndexSchema(**video_obj).dict()

In [ ]:
playlist_obj = dict(Playlist.objects.first())
playlist_obj

In [ ]:
class PlaylistIndexSchema2(BaseModel):
    objectID: uuid.UUID = Field(alias='db_id')
    objectType: str = "Playlist"
    title: Optional[str]
    path: str = Field(default='/')
    
    @root_validator
    def set_defaults(cls, values):
        objectID = values.get('objectID')
        values['objectID'] = str(objectID)
        values['path'] = f"/playlists/{objectID}"
        return values

In [ ]:
# json.loads(PlaylistIndexSchema(**playlist_obj).json())

In [ ]:
PlaylistIndexSchema(**playlist_obj).dict()

In [ ]:
video_q = [dict(x) for x in Video.objects.all()]
videos_dataset = [VideoIndexSchema(**x).dict() for x in video_q]
videos_dataset

In [ ]:
playlist_q = [dict(x) for x in Playlist.objects.all()]
playlists_dataset = [PlaylistIndexSchema(**x).dict() for x in playlist_q]
playlists_dataset

In [ ]:
dataset = playlists_dataset + videos_dataset

In [ ]:
dataset

In [ ]:
index = get_index()

In [ ]:
# dir(index)

In [ ]:
# idx_response = index.save_objects(dataset).wait()
# len(idx_response[0]['objectIDs'])

updated_count = update_index()
updated_count

In [ ]:
index.search("OpenCV Is Awesome!")